In [4]:
import pandas as pd
import cobra
import memote
import os 

In [6]:
os.chdir('/Users/benjaminreyes/Desktop/Masterarbeit/neutrophil_modeling/neutrophil-modeling/curation')

In [8]:
mitocore0= cobra.io.read_sbml_model('Mitocore_0.xml')
mitocore1= cobra.io.read_sbml_model('Mitocore_Curated_Emanuel.xml')
mitocore2= cobra.io.read_sbml_model('Cleaned_Mitocore_changed_metabolites_metanetx.chemical.xml')

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_10fthf_m "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_m "10-Formyltetrahydrofolate">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_13dpg_c "3-Phospho-D-glyceroyl phosphate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_13dpg_c "3-Phospho-D-glyceroyl phosphate">
Use of CHARGE i

In [5]:
def visualization_model_annotations(model, component):
    """"
    model: COBRApy model object (e.g., mitocore)
    component: str, ('reactions', 'metabolites', 'genes')
    """
    
    mitocore= cobra.io.read_sbml_model(model)
    counter=0
    for comp in mitocore.__getattribute__(component):
        comp_complete= comp.__dict__
        annot_dict= comp.__dict__.get( '_annotation')
        counter+=1
        print(annot_dict)

def visualization_model_general(model, component):
    """"
    model: COBRApy model object (e.g., mitocore)
    component: str, ('reactions', 'metabolites', 'genes')
    """
    
    mitocore= cobra.io.read_sbml_model(model)
    for comp in mitocore.__getattribute__(component):
        comp_complete= comp.__dict__
        print(comp_complete)

def visualization_model_notes(model, component):
    """"
    model: COBRApy model object (e.g., mitocore)
    component: str, ('reactions', 'metabolites', 'genes')
    """
    
    mitocore= cobra.io.read_sbml_model(model)
    counter=0
    for comp in mitocore.__getattribute__(component):
        comp_complete= comp.__dict__
        notes_dict= comp.__dict__.get( 'notes')
        counter+=1
        print(notes_dict)

In [20]:
#visualization_model_notes('Mitocore_0.xml', 'genes')
visualization_model_general('Mitocore_Curated_Emanuel.xml', 'reactions')

{'_id': 'EX_2hb_e', 'name': 'EX_2hb_e', 'notes': {}, '_annotation': {}, '_gpr': cobra.core.gene.GPR(''), 'subsystem': '', '_genes': set(), '_metabolites': {<Metabolite 2hb_e at 0x164636a90>: -1.0}, '_model': <Model S1SBMLmodel at 0x16d5c3a90>, '_lower_bound': -1000.0, '_upper_bound': 1000.0}
{'_id': 'EX_ac_e', 'name': 'EX_ac_e', 'notes': {}, '_annotation': {}, '_gpr': cobra.core.gene.GPR(''), 'subsystem': '', '_genes': set(), '_metabolites': {<Metabolite ac_e at 0x169dee450>: -1.0}, '_model': <Model S1SBMLmodel at 0x16d5c3a90>, '_lower_bound': -1000.0, '_upper_bound': 1000.0}
{'_id': 'EX_acac_e', 'name': 'EX_acac_e', 'notes': {}, '_annotation': {}, '_gpr': cobra.core.gene.GPR(''), 'subsystem': '', '_genes': set(), '_metabolites': {<Metabolite acac_e at 0x28b116f50>: -1.0}, '_model': <Model S1SBMLmodel at 0x16d5c3a90>, '_lower_bound': -1000.0, '_upper_bound': 1000.0}
{'_id': 'EX_akg_e', 'name': 'EX_akg_e', 'notes': {}, '_annotation': {}, '_gpr': cobra.core.gene.GPR(''), 'subsystem': '',

In [ ]:
visualization_model_annotations('Mitocore_Curated_Emanuel.xml', 'reactions')

In [ ]:

def get_model_ids_for_databases(model, component_type, dict, databases):
    """
    Extract ids from model annotations for various databases across a specified component type.
    
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: str, one of 'reactions', 'metabolites', 'genes'
    - dict: dictionary where database ids are located ('annotations', 'notes')
    - databases: list of str, e.g., ['kegg', 'bigg', 'metanetx', 'uniprot'].
    
    Returns:
    - DataFrame with component ID and extracted database ids.
    """
    # Initialize dictionary
    ids_dict = {'model_id': []}

    counters= {db:0 for db in databases}

    for db in databases: 
        ids_dict[db] = []

    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type)

    for comp in components:
        ids_dict['model_id'].append(comp.id)

        for db in databases:
            value = None
            annotation = getattr(comp, dict, {})

            # Use several naming conventions
            for key in [db, 
                        f"{db}.reaction", 
                        f"{db}.compound", 
                        f"{db}.metabolite", 
                        f"{db}.chemical", 
                        f"{db}.genes",
                        f"{db}.gene",
                        f"{db} id",
                        f"{db.upper()} ID",
                        f"{db.lower()} ID"
                        ]:
                if key in annotation and annotation[key] not in [None, '', 'N/A']:
                    counters[db] += 1
                    value = annotation[key]
                    break


            ids_dict[db].append(value)
            #pd.DataFrame(ids_dict)
    
    return counters

def get_model_ids_for_attr(model, component_type, attr_name):
    """
    Extract attribute in the model in the target core-component.
    
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: Core component of GEMs, str, one of 'reactions', 'metabolites', 'genes'
    - attr_name: name of the dictionary where target object is located ('notes', '_gpr')

    
    Returns:
    - DataFrame with component ID and extracted database ids.
    """
    # Initialize dictionary
    ids_dict = {'model_id': [], attr_name:[]}

    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type)

    for comp in components:
        ids_dict['model_id'].append(comp.id)
        value = comp.__dict__.get(attr_name, None)
        ids_dict[attr_name].append(value)

    return pd.DataFrame(ids_dict)

# Metabolites

In [8]:
mitocore0_react= get_model_ids_for_databases(mitocore0, 'metabolites', 'notes', ['KEGG', 'RECON2', 'metanetx', 'Human1'])
mitocore1_react= get_model_ids_for_databases(mitocore1, 'metabolites', '_annotation', ['kegg','bigg', 'metanetx','Human1'])
mitocore2_react= get_model_ids_for_databases(mitocore2, 'metabolites', '_annotation', ['kegg','bigg', 'metanetx','Human1'])


print('Mitocore original', mitocore0_react)
print('Mitocore 1st curation', mitocore1_react)
print('Mitocore 2nd curation', mitocore2_react)

Mitocore original {'KEGG': 436, 'RECON2': 436, 'metanetx': 0, 'Human1': 0}
Mitocore 1st curation {'kegg': 436, 'bigg': 325, 'metanetx': 0, 'Human1': 0}
Mitocore 2nd curation {'kegg': 436, 'bigg': 325, 'metanetx': 436, 'Human1': 421}


# Reactions

In [9]:
mitocore0_react= get_model_ids_for_databases(mitocore0, 'reactions', 'notes', ['KEGG', 'Recon2','EC Number', 'Human1'])
mitocore1_react= get_model_ids_for_databases(mitocore1, 'reactions', '_annotation', ['kegg', 'bigg', 'ec-code', 'metanetx', 'Human1'])
mitocore2_react= get_model_ids_for_databases(mitocore2, 'reactions', '_annotation', ['kegg', 'bigg', 'ec-code', 'metanetx', 'Human1'])


print('Mitocore original', mitocore0_react)
print('Mitocore 1st curation', mitocore1_react)
print('Mitocore 2nd curation', mitocore2_react)

Mitocore original {'KEGG': 328, 'Recon2': 432, 'EC Number': 328, 'Human1': 0}
Mitocore 1st curation {'kegg': 324, 'bigg': 0, 'ec-code': 322, 'metanetx': 0, 'Human1': 0}
Mitocore 2nd curation {'kegg': 328, 'bigg': 432, 'ec-code': 323, 'metanetx': 335, 'Human1': 395}


# Genes

In [10]:
mitocore0_react= get_model_ids_for_databases(mitocore0, 'genes', 'notes', ['Uniprot', 'kegg', 'Human1'])
mitocore1_react= get_model_ids_for_databases(mitocore1, 'genes', '_annotation', ['uniprot','kegg', 'Human1'])
mitocore2_react= get_model_ids_for_databases(mitocore2, 'genes', '_annotation', ['uniprot', 'kegg', 'Human1'])


print('Mitocore original', mitocore0_react)
print('Mitocore 1st curation', mitocore1_react)
print('Mitocore 2nd curation', mitocore2_react)

Mitocore original {'Uniprot': 0, 'kegg': 0, 'Human1': 0}
Mitocore 1st curation {'uniprot': 387, 'kegg': 0, 'Human1': 0}
Mitocore 2nd curation {'uniprot': 595, 'kegg': 587, 'Human1': 595}


# --- ec-codes -------

In [22]:
curation_0_ec = []

for reaction in mitocore0.reactions:
    annotations = getattr(reaction, 'notes', {})
    ec_data = annotations.get('EC Number', [])

    # Normalize: ensure ec_data is always a list
    if isinstance(ec_data, str):
        ec_data = [ec_data]

    # iterate over str (ec-code) in a list made of str (ec_data)
    for ec_code in ec_data:
        if isinstance(ec_code, str):
            curation_0_ec.append(ec_code)

print('mitocore original:', len(curation_0_ec), curation_0_ec)

curation_1_ec = []

for reaction in mitocore1.reactions:
    annotations = getattr(reaction, '_annotation', {})
    ec_data = annotations.get('ec-code', [])

    # Normalize: ensure ec_data is always a list
    if isinstance(ec_data, str):
        ec_data = [ec_data]

    # iterate over str (ec-code) in a list made of str (ec_data)
    for ec_code in ec_data:
        if isinstance(ec_code, str):
            curation_1_ec.append(ec_code)

print('mitocore 1 curation:', len(curation_1_ec), curation_1_ec)


curation_2_ec=[]

for reaction in mitocore2.reactions:
    annotations = getattr(reaction, '_annotation', {})
    ec_data = annotations.get('ec-code', [])

    # Normalize: ensure ec_data is always a list
    if isinstance(ec_data, str):
        ec_data = [ec_data]
        
    # iterate over str (ec-code) in a list made of str (ec_data)
    for ec_code in ec_data:
        if isinstance(ec_code, str):
            curation_2_ec.append(ec_code)

print('mitocore curation 2:', len(curation_2_ec), curation_2_ec)

mitocore original: 339 ['2.7.1.1 or 2.7.1.2', '3.1.3.9', '5.3.1.9', '2.7.1.11', '3.1.3.11', '4.1.2.13', '5.3.1.1', '1.2.1.12', '2.7.2.3', '5.4.2.1', '4.2.1.11', '2.7.1.40', '2.7.1.40', '4.1.1.32', '1.1.1.27', '1.1.1.49', '3.1.1.31', '1.1.1.44', '5.3.1.6', '5.1.3.1', '2.2.1.1', '2.2.1.2', '2.2.1.1', '1.2.4.1, 2.3.1.12, 1.8.1.4', '2.3.3.1', '4.2.1.3', '1.1.1.41', '1.1.1.42', '1.2.4.2, 1.8.1.4, 2.3.1.61', '6.2.1.4', '6.2.1.5', '4.2.1.2', '1.1.1.37', '1.6.5.3, 1.6.5.8, 1.6.5.9', '1.3.5.1', '1.10.2.2', '1.9.3.1', '3.6.3.14, 3.6.3.10, 3.6.3.6', '4.1.1.32', '6.4.1.1', '1.1.1.40', '1.1.1.38', '2.6.1.2', '2.3.3.8', '2.7.4.6', '1.6.1.2', '2.7.4.3', '1.1.1.40', '2.6.1.2', '2.7.4.6', '4.2.1.2', '2.7.4.3', '1.1.1.42', '4.2.1.3', '2.3.3.8', '2.6.1.1', '1.1.1.37', 'N/A', 'N/A', '2.6.1.1', '1.1.1.8', '1.1.5.3', '6.2.1.3', '2.3.1.21', '3.6.1.1', 'N/A', '2.3.1.21', '3.6.1.1', '3.1.2.2', '1.3.8.8, 1.5.5.1', '1.3.1.38', '4.2.1.74, 1.1.1.211, 2.3.1.16', '1.3.8.8, 1.5.5.1', '1.3.1.38', '4.2.1.74, 1.1.1.211,

# ---- GPR rules ----


In [1]:
def get_model_ids_for_attr(model, component_type, attr_name):
    """
    Extract attribute in the model in the target core-component.
    
    Parameters:
    - model: COBRApy model object (mitocore)
    - component_type: Core component of GEMs, str, one of 'reactions', 'metabolites', 'genes'
    - attr_name: name of the dictionary where target object is located ('notes', '_gpr')

    
    Returns:
    - DataFrame with component ID and extracted database ids.
    """
    # Initialize dictionary
    ids_dict = {'model_id': [], attr_name:[]}

    # Access model component (e.g., reactions, genes)
    components = getattr(model, component_type)

    for comp in components:
        ids_dict['model_id'].append(comp.id)
        value = comp.__dict__.get(attr_name, None)
        ids_dict[attr_name].append(value)

    return pd.DataFrame(ids_dict)

In [12]:
gpr_mit_0=get_model_ids_for_attr(mitocore0, 'reactions', '_gpr')
gpr_mit_1=get_model_ids_for_attr(mitocore1, 'reactions', '_gpr')
gpr_mit_2= get_model_ids_for_attr(mitocore2, 'reactions', '_gpr')

print('GPR Mitocore 0:', gpr_mit_0[150:155])
print('GPR Mitocore 1:', gpr_mit_1[150:155])
print('GPR Mitocore 2:', gpr_mit_2[150:155])

GPR Mitocore 0:     model_id                                 _gpr
150    r0638                      ENSG00000116353
151    r0660                      ENSG00000127884
152    r0722                      ENSG00000138796
153    r0724                      ENSG00000167315
154    r1451  ENSG00000117054 and ENSG00000171503
GPR Mitocore 1:     model_id                                 _gpr
150    r0638                      ENSG00000116353
151    r0660                      ENSG00000127884
152    r0722                      ENSG00000138796
153    r0724                      ENSG00000167315
154    r1451  ENSG00000117054 and ENSG00000171503
GPR Mitocore 2:     model_id                                               _gpr
150    r0638                                    ENSG00000116353
151    r0660  ENSG00000127884 or (ENSG00000084754 and ENSG00...
152    r0722  ENSG00000138796 or (ENSG00000060971 and ENSG00...
153    r0724  ENSG00000167315 or (ENSG00000060971 and ENSG00...
154    r1451  (ENSG00000117054 a

In [ ]:
# Split GPR on 'or' 
def gpr_counte(df, model):

    gpr_absolute= []

    for index, row in df.iterrows():
        gpr = row['_gpr']
        elements_in_gpr = str(gpr).split(' or ')

        for element in elements_in_gpr:
            if element not in [None, 'N/A','']:
                gpr_absolute.append(element)
    print(gpr_absolute)
    return print(f" {model}, has {len(gpr_absolute)} GPRs" )

In [38]:
gpr_0=gpr_counte(gpr_mit_0, 'Mitocore0')
gpr_0=gpr_counte(gpr_mit_1, 'Mitocore1')
gpr_0=gpr_counte(gpr_mit_2, 'Mitocore2')


['ENSG00000156515', 'ENSG00000159399', 'ENSG00000160883', 'ENSG00000106633', 'ENSG00000131482', 'ENSG00000105220', 'ENSG00000152556', 'ENSG00000165140', 'ENSG00000130957', 'ENSG00000149925', 'ENSG00000109107', 'ENSG00000111669', 'ENSG00000111640', 'ENSG00000102144', 'ENSG00000171314', 'ENSG00000164708', 'ENSG00000074800', 'ENSG00000111674', 'ENSG00000108515', 'ENSG00000067225', 'ENSG00000067225', 'ENSG00000124253', 'ENSG00000111716', 'ENSG00000160211', 'ENSG00000130313', 'ENSG00000142657', 'ENSG00000153574', 'ENSG00000235376', 'ENSG00000163931', 'ENSG00000177156', 'ENSG00000163931', 'ENSG00000131828 and ENSG00000168291 and ENSG00000150768 and ENSG00000091140', 'ENSG00000062485', 'ENSG00000100412', 'ENSG00000166411 and ENSG00000101365 and ENSG00000067829', 'ENSG00000182054', 'ENSG00000105953 and ENSG00000119689 and ENSG00000091140', 'ENSG00000163541 and ENSG00000172340', 'ENSG00000136143', 'ENSG00000091483', 'ENSG00000146701', 'ENSG00000115286 and ENSG00000110717 and ENSG00000178127 and

In [ ]:
mitocore =cobra.io.read_sbml_model('Mitocore_changed_reactions_ec-code.xml')

#-------------Genes GPR extraction-----------------
gene_list_gpr=[]
pattern= r'ENSG\d{11}'

for reaction in mitocore.reactions:
    gpr= reaction.gene_reaction_rule.strip()
    matches = re.findall(pattern, gpr)
    if matches:
        gene_list_gpr.extend(matches) # .extend() add the objects as strings and not like a list as .append() does

gene_list_gpr = list(gene_list_gpr)
print('genes from GPRs:', len(gene_list_gpr), gene_list_gpr)
gene_list_gpr = list(set(gene_list_gpr))
print('genes from GPRs after removing duplicates:', len(gene_list_gpr), gene_list_gpr)

#---------------Genes Mitocore 'genes' compartment--------------
gene_list_mito=[]

for gene in mitocore.genes:
    id= gene.id
    gene_list_mito.append(id)

gene_list_mito = list(set(gene_list_mito))
print('genes in mitocore.genes:',len(gene_list_mito), gene_list_mito )  

#---------check how many genes from GPRs are in Genes compartment------------

genes_contained=[]
genes_not_contained=[]

for gene in gene_list_gpr:
    if gene in gene_list_mito:
        genes_contained.append(gene)
    else: 
        genes_not_contained.append(gene)

print(len(genes_not_contained))


# GPR comparison Mitocore original, Preliminary curation, Alignment to Human1

In [ ]:
# extract GPRs for original // curation1 // final curation into a csv file 
# for visualization of the code because memote does not show GPRs
mitocore0= cobra.io.read_sbml_model('Mitocore_0.xml')
mitocore1= cobra.io.read_sbml_model('Mitocore_Curated_Emanuel.xml')
mitocore2= cobra.io.read_sbml_model('Mitocore_GPR_Cleaned.xml')

gpr_mitocore_0= get_model_ids_for_attr(mitocore0, 'reactions', '_gpr')
gpr_mitocore_1= get_model_ids_for_attr(mitocore1, 'reactions', '_gpr')
gpr_mitocore_2= get_model_ids_for_attr(mitocore2, 'reactions', '_gpr')


# Merge mito0 and mito1 (not possible to merge)
merged_01 = pd.merge(
    gpr_mitocore_0, 
    gpr_mitocore_1, 
    on='model_id', 
    suffixes=('_original', '_emanuel')
)

# Merge merged_01 with mito2
gpr_mitocore = pd.merge(
    merged_01, 
    gpr_mitocore_2, 
    on='model_id'
    ,suffixes=('_cleaned','' )
)

gpr_mitocore.to_csv('GPR_comparison.csv', index=False)